# Extracting table header as list

In [2]:
import collections
collections.Callable = collections.abc.Callable

In [3]:
from bs4 import BeautifulSoup, NavigableString
from html_table_extractor.extractor import Extractor
import pandas as pd
import os
import re

In [3]:
path = "/home/riya/Downloads/ADT_2019-20/Output_html_Chapter10/kaggle/working/out/output/"

In [4]:
os.chdir(path)

In [5]:
os.listdir()

['output_page_18.html',
 'output_page_15.html',
 'output_page_10.html',
 'output_page_16.html',
 'output_page_1.html',
 'output_page_2.html',
 'output_page_5.html',
 'output_page_11.html',
 'output_page_8.html',
 'output_page_19.html',
 'output_page_3.html',
 'output_page_13.html',
 'output_page_4.html',
 'output_page_14.html',
 'output_page_9.html',
 'output_page_12.html',
 'output_page_17.html',
 'output_page_7.html',
 'output_page_6.html']

In [39]:
output_list = []

In [40]:
output_list

[]

In [41]:
allowed_two_letter_words = {'sl','sc', 'st', 'cc', 'tp', 'tb', 'ts', 'tg', 'km', 'gm', 'an', 'at', 'by', 'do', 'go', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'of', 'on', 'or', 'so', 'to', 'up', 'us', 'mm'}

def replace_td_content(tag):
    if isinstance(tag, NavigableString):
        return  # Skip strings, as they don't have 'contents'

    if tag.name == 'td':
        # Extract only meaningful English words and numbers
        valid_characters = set("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-.%")
        filtered_content = ''.join(char if char in valid_characters else ' ' for char in tag.get_text())
        
        # Remove extra spaces and words with less than 3 characters
        cleaned_content = ' '.join(word for word in filtered_content.split() if len(word) >= 3 or word.isdigit() or word.lower() in allowed_two_letter_words or word=='%')
        
        # Set the cleaned content as the tag's string
        tag.string = cleaned_content
    elif tag.contents:
        for child in tag.contents:
            replace_td_content(child)


In [42]:
def process_table_cells(target_table):
    if not target_table:
        print("Table not found in the HTML content.")
        return

    flag1 = 0
    flag2 = 0

    # Loop through each row in the table
    for row in target_table.find_all('tr'):
        # Loop through each cell in the row
        if(flag1 == 1 and flag2 == 1):
            row.decompose()
        else : 
            for cell in row.find_all(['td', 'th']):
                # Process the cell content (you can modify this part based on your requirements)
                cell_content = cell.get_text(strip=True)
                if(cell_content == '1'):
                    flag1 = 1
                if(cell_content == '2'):
                    flag2 = 1        
            

In [43]:
# iterate through all file 
for file in os.listdir(): 
    # Check whether file is in text format or not 
    if file.endswith(".html"): 
        file_path = f"{path}{file}"
        with open(file_path) as fp:
            soup = BeautifulSoup(fp, "html.parser")
            
            for tag in soup.find_all(attrs={'style': True}):
                tag.attrs.pop('style')
            
            table_tag = soup.find('table')
            
            if table_tag is not None:
                replace_td_content(table_tag)

                process_table_cells(table_tag)

                extractor1 = Extractor(table_tag)
                extractor1.parse()
                output_list.append(extractor1.return_list())
            else:
                print(file_path)

In [44]:
output_list

[[['10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20'],
  ['No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20'],
  ['No',
   'Districts',
   'Government Colleges',
   'Government Colleges',
   'Government Colleges',
   'Government Colleges',
   'Private Colleges Aided Unaided',
   'Private Colleges Aided Unaided',
   'Private Colleges Aided Unaided',
   'Priv

In [45]:
def contain_table_id(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'(?<!\S)(?:' + '(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})' + r')(?!\S)', flags=re.IGNORECASE)

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    if len(matches) == 0:
        return False
    return True

In [46]:
def are_all_same_id(lst):
    if not lst:
        return False  # An empty list cannot have all same elements.
    first_element = lst[0]
    if first_element != '' and contain_table_id(first_element):
        return all(element == first_element for element in lst[:-1])
    return False

def are_all_same_name(lst):
    if not lst:
        return False  # An empty list cannot have all same elements.
    first_element = lst[0]
    if first_element != '':
        return all(element == first_element for element in lst[:-1])
    return False

In [47]:
def check_for_table(item):
    i=0
    new_item = item
    for k in range(len(item)):
        id_list=[]
        if are_all_same_id(item[k]):
            if i<1:
                i = i+1
            elif i==2:
                return item
        elif i==1 and are_all_same_name(item[k]):
            return item
        elif i==1 and not are_all_same_name(item[k]):
            n_lst = item[k-1]
            new_item.insert(0,n_lst)
            i = i+1
            return new_item
        else:
            new_item.remove(item[k])
            return check_for_table(new_item)

In [48]:
corrected_list=[]
error_list=[]
for inner_list in output_list:
    initial_val = inner_list.copy()
    item = check_for_table(inner_list)
    if item is not None and len(item)>0:
        corrected_list.append(item)
    else:
        error_list.append(initial_val)

In [49]:
corrected_list

[[['10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20',
   '10.14 2019 20'],
  ['No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20',
   'No. of Polytechnic Colleges and Students 2019 20'],
  ['No',
   'Districts',
   'Government Colleges',
   'Government Colleges',
   'Government Colleges',
   'Government Colleges',
   'Private Colleges Aided Unaided',
   'Private Colleges Aided Unaided',
   'Private Colleges Aided Unaided',
   'Priv

In [50]:
error_list

[]

In [91]:
for item in error_list:
    corrected_list.remove(item)

In [51]:
list_to_add = [[['5.1',
   '5.1',
   '5.1',
   '5.1',
   '5.1',
   '5.1',
   '5.1',
   '5.1',
   '5.1',
   '5.1',
   '5.1'],
  ['5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers',
   '5.1 20 12 Livestock Census 2012 In Numbers'],
  ['',
   '',
   'Cattle',
   'Cattle',
   'Cattle',
   'Cattle',
   'Bufaloes',
   'Sheep',
   'Sheep',
   'Sheep',
   'Sheep'],
  ['',
   '',
   'Indigenous Breed',
   'Exotic Breed',
   'Cross Breed',
   'Total',
   'Bufaloes',
   'Indigenous Breed',
   'Exotic Breed',
   'Cross Breed',
   'Total'],
  ['1', '2', '790', '791', '792', '793', '794', '795', '796', '797', '798']],
 [['5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012'],
   ['5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012',
   '5.1 Livestock Census 2012'],
  ['',
   '',
   'Rabbits',
   'Dogs',
   'Others',
   'Total Livestock',
   'Total Poultry',
   '% of Total Livestock to state total'],
  ['1', '2', '804', '805', '806', '807', '808', '809']]]

In [52]:
corrected_list.extend(list_to_add)

In [51]:
len(output_list)

19

In [52]:
len(corrected_list)

19

In [53]:
len(error_list)

0

In [54]:
def add_space_before_year(input_string):
    # Regular expression to find a year in 'yyyy' format with an optional alphabet character before it
    pattern = re.compile(r'([a-zA-Z])?(\d{4})')
    
    # Function to add space between alphabet character and year
    def replace(match):
        alphabet_part, year_part = match.groups()
        if alphabet_part:
            return f'{alphabet_part} {year_part}'
        else:
            return year_part

    # Using re.sub() to apply the replacement function to the input string
    result = pattern.sub(replace, input_string)

    return result

In [55]:
def remove_before_all_occurrence(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    for i in range(len(matches)):
        second_occurrence_index = matches[i].end()
        input_string = input_string[second_occurrence_index:]

    return input_string.strip()

In [56]:
def remove_after_all_occurrence(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = list(pattern.finditer(input_string))

    # If there are at least two occurrences of the pattern, remove everything before the second occurrence
    for i in range(len(matches)):
        second_occurrence_index = matches[i].end()
        input_string = input_string[:second_occurrence_index]

    return input_string.strip()

In [57]:
def convert_uppercase_to_camel_case(input_string):
    words = input_string.split()
    result_words = [word.capitalize() if word.isupper() else word for word in words]
    result_string = ' '.join(result_words)
    return result_string

In [58]:
def add_space_before_camelcase(input_string):
    # Use regular expression to find camel case words
    camelcase_pattern = re.compile(r'(?<=[a-z])(?=[A-Z])')
    result_string = camelcase_pattern.sub(' ', input_string)
    return result_string

In [59]:
def remove_extra_spaces(input_string):
    # Use regular expression to replace consecutive white spaces with a single space
    cleaned_string = re.sub(r'\s+', ' ', input_string)
    return cleaned_string.strip()

In [60]:
def add_space_before_word_with_condition(input_string):
    # Use regular expression to find words preceded by a number or special character
    pattern = re.compile(r'(?<=[%])\b(\w+)\b', flags=re.IGNORECASE)

    # Using re.sub() to add space before matched words
    result_string = pattern.sub(lambda x: ' ' + x.group(1), input_string)

    return result_string

In [61]:
def replace_words_with_blank(input_string, words_to_replace):
    # Constructing a regular expression pattern to match any of the words in the set
    # pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in words_to_replace) + r')\b', flags=re.IGNORECASE)
    pattern = re.compile(r'(?<!\S)(?:' + '|'.join(re.escape(word) for word in words_to_replace) + r')(?!\S)', flags=re.IGNORECASE)

    # Using re.sub() to replace matched words with blank
    result = pattern.sub('', input_string)

    return result.strip()

In [62]:
words_to_replace = ['1 2', '6 14', '2009 2019', '1961 201', '-2011', '0-6 2011', '0-6 2001', '0-6 2001 2011', '2001 and 2011', '20 01', '2011 2021', '31.03', '.CA', '.Dg', '.G.S', '.Jfl', '.Q.', '.ffl', '1 2', '1-10', '2 01', '2006 jAz 2016', '2011 twAi rAi AaP fiU', '2011 twAi zsj wAi EAzs %', '20112.2', '20112.6', '2011g', '24 7', '31.03.2020', '31.3.202', '31.3.2020', '31.3.2020.', '33 jAz 6', '6 jAz 1', '6 jAz 14', '6 wAU', 'A.ffl', 'A.g', 'AdP', 'Adg', 'Affl', 'Afi', 'Afli', 'Aid', 'Arg', 'Azs', 'CAU', 'CAZ', 'CAv', 'CAw', 'CAz', 'Cfi', 'CrP', 'Dfi', 'Dfl', 'E.J .n. .Dg .Jfl .JA.', 'EAf', 'Efi', 'Efl', 'JfiP', 'Jfl', 'KfiQ', 'MlM', 'Pfi', 'QAi', 'Raing', 'Rfl', 'SQA', 'Three', 'VAi', 'dfl', 'eng cy.U', 'ffl', 'fiAi', 'fiP', 'fiQ', 'fiQAi', 'fiU', 'fig', 'fl.', 'gLi', 'gMl', 'jAU', 'jAi', 'lion nit', 'ofD', 'rAi', 'tAwAi', 'twAi', 'tzx', 'wAi', 'tzs']

In [63]:
def replace_words(input_string, word_replacements):
    # Constructing a regular expression pattern to match whole words in the set
    # pattern = re.compile(r'\b(?:' + '|'.join(re.escape(word) for word in word_replacements.keys()) + r')\b')
    pattern = re.compile(r'(?<!\S)(?:' + '|'.join(re.escape(word) for word in word_replacements.keys()) + r')(?!\S)')

    # Using re.sub() to replace matched words with their corresponding replacements
    result = pattern.sub(lambda x: word_replacements.get(x.group(), x.group()), input_string)

    return result

In [64]:
word_replacements = {"2019 20" : "2019-20", "from1st" : "from 1st", "on" : "on 31-03-2020", "2011 Literates" : "Literates", "2011 Literacy" : "Literacy", "Dropoutratio" : "Drop out ratio", "1 2nd" : "1st and 2nd", "ofstudents" : "of students", "No. of" : "No.of"}

In [65]:
def process_data(input_data):
    for k in range(len(input_data)):
        for i in range(len(input_data[k])):
            for j in range(len(input_data[k][i])):
                #print(k,i,j,input_data[k][i][j])

                input_data[k][i][j] = add_space_before_year(input_data[k][i][j])
                input_data[k][i][j] = replace_words_with_blank(input_data[k][i][j], words_to_replace)
                input_data[k][i][j] = replace_words(input_data[k][i][j], word_replacements)
                input_data[k][i][j] = add_space_before_camelcase(input_data[k][i][j])
                input_data[k][i][j] = convert_uppercase_to_camel_case(input_data[k][i][j])
                input_data[k][i][j] = remove_extra_spaces(input_data[k][i][j])
                input_data[k][i][j] = add_space_before_word_with_condition(input_data[k][i][j])

In [66]:
process_data(corrected_list)
corrected_list

[[['10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20',
   '10.14 2019-20'],
  ['No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20'],
  ['No',
   'Districts',
   'Government Colleges',
   'Government Colleges',
   'Government Colleges',
   'Government Colleges',
   'Private Colleges Aided Unaided',
   'Private Colleges Aided Unaided',
   'Private Colleges Aided Unaided',
   'Private Colleg

In [67]:
#transpose the inner list
transp_list = [[list(pair) for pair in zip(*sublist)] for sublist in corrected_list]
transp_list

[[['10.14 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'No',
   'No',
   'No',
   '1'],
  ['10.14 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'Districts',
   'Districts',
   'Districts',
   '2'],
  ['10.14 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'Government Colleges',
   'No.of Colleges',
   'No.of Colleges',
   '1135'],
  ['10.14 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'Government Colleges',
   'Students from 1st year to final year',
   'Boys',
   '1136'],
  ['10.14 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'Government Colleges',
   'Students from 1st year to final year',
   'Girls',
   '1137'],
  ['10.14 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'Government Colleges',
   'Students from 1st year to final year',
   'Total',
   '1138'],
  ['10.14 2019-20',
   'No.of Polytechnic Colleges and Students 2019-20',
   'Private Colleges Aided Unai

In [68]:
def find_table_details(input_string):
    # Regular expression to match numbers with dot similar to dd.dd, d.d, d.dd, or dd.d
    pattern = re.compile(r'\b(\d\.\d|\d{2}\.\d|\d\.\d{2}|\d{2}\.\d{2})\b')

    # Find all occurrences of the pattern in the input string
    matches = pattern.finditer(input_string)

    # Initialize variables to store the matches
    match_positions = []

    # Store the positions of the matches
    for match in matches:
        match_positions.append(match.start())

    # If there is at least one occurrence of the pattern
    if match_positions:
        table_id_index = match_positions[0]
        input_string = input_string[table_id_index:]  
        space_index = input_string.find(' ') 
        table_id = input_string[table_id_index:space_index].strip()
        table_name = input_string[space_index:].strip()
        return table_id, table_name    
    else:
        return None, None


In [69]:
def convert_to_number(string):
    try:
        number = int(string)
    except ValueError:
        try:
            number = float(string)
        except ValueError:
            print(string)
            raise ValueError("Input string is not a valid number")
    return number

In [70]:
chapter_id = 10
chapter_name = "Education"
#processing each column
def process_dictionary(input_data):
    df = pd.DataFrame(columns = ['Attr_id', 'Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description'])
    for inner_list in input_data:
        table_id = remove_after_all_occurrence(inner_list[0][0])
        table_name = remove_before_all_occurrence(inner_list[0][1])
        previd=0
        for data_list in inner_list:
            i=0
            attr_id=0
            description = ''
            prev = ''
            for data in reversed(data_list):
                if data == '1' or data == '2':
                    break
                i = i+1;
                if i == 1 and data!='' and (previd==0 or convert_to_number(data)!=previd):
                    attr_id = convert_to_number(data)
                    previd = convert_to_number(data)
                if i > 1 and i < len(data_list)-1:
                    if description == '':
                        description = data
                        prev = data
                    else : 
                        if prev != data and data!='':
                            description = ','.join([description, data]) #description + ' ' + data
                            prev = data
            if attr_id != 0 and description != "" :
                row = {'Attr_id' : attr_id, 'Chapter_id' : chapter_id, 'Chapter_name' : chapter_name, 'Table_id' : table_id, 'Table_name' : table_name, 'Description' : remove_extra_spaces(description)}
                df.loc[len(df)] = row
    df = df.sort_values(by=['Attr_id'])
    df.to_csv('/home/riya/Downloads/ADT_2019-20/Output_html_Chapter10/Chapter10_Attributes.csv', sep=';', index=False)
    df.to_csv('/home/riya/Downloads/ADT_2019-20/Dictionary/Chapter10_Attributes.csv', sep=';', index=False)
                                            

In [71]:
process_dictionary(transp_list)